<a href="https://colab.research.google.com/github/7pupuwen/PCB_QDETECTION/blob/main/PCB_Detection_feature_data_pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd

# Set the label files
train_labels_path = "C:/Users/wenwe/Desktop/Lab_QC/Wendy/pcb-defect-dataset/train/labels"
val_labels_path = "C:/Users/wenwe/Desktop/Lab_QC/Wendy/pcb-defect-dataset/val/labels"
test_labels_path = "C:/Users/wenwe/Desktop/Lab_QC/Wendy/pcb-defect-dataset/test/labels"

# Definition of convert label to csv
def convert_labels_to_csv(labels_path, output_csv):
    data = []

    # read every .txt file
    for file in os.listdir(labels_path):
        if file.endswith(".txt"):
            file_path = os.path.join(labels_path, file)
            image_name = file.replace(".txt", ".jpg")  # The file name corresponding to the image
            with open(file_path, "r") as f:
                lines = f.readlines()  #A .txt may have multiple lines of tags

                for line in lines:
                    values = line.strip().split()
                    class_id = int(values[0])  # Defect category
                    x_center, y_center, width, height = map(float, values[1:])  # Analyze numerical values

                    # Save to DataFrame
                    data.append([image_name, class_id, x_center, y_center, width, height])

    # create DataFrame
    df = pd.DataFrame(data, columns=["image_name", "class_id", "x_center", "y_center", "width", "height"])

    # Save as CSV
    df.to_csv(output_csv, index=False)

    print(f"{output_csv} convert finish！")
    print(df.head(10))  # show the first 10 data

# covert `train`、`val`、`test` labels
convert_labels_to_csv(train_labels_path, "train_labels.csv")
convert_labels_to_csv(val_labels_path, "val_labels.csv")
convert_labels_to_csv(test_labels_path, "test_labels.csv")

train_labels.csv convert finish！
                           image_name  class_id  x_center  y_center   width  \
0  light_01_missing_hole_01_1_256.jpg         2    0.4350    0.7008  0.0700   
1  light_01_missing_hole_01_2_256.jpg         2    0.6367    0.0508  0.0667   
2  light_01_missing_hole_01_2_256.jpg         2    0.7575    0.5333  0.0717   
3  light_01_missing_hole_02_1_256.jpg         2    0.5633    0.2783  0.0667   
4  light_01_missing_hole_02_1_256.jpg         2    0.3208    0.8733  0.0383   
5  light_01_missing_hole_03_1_256.jpg         2    0.6258    0.5350  0.0483   
6  light_01_missing_hole_03_1_256.jpg         2    0.0292    0.6317  0.0517   
7  light_01_missing_hole_04_1_256.jpg         2    0.3808    0.8242  0.0483   
8  light_01_missing_hole_04_2_256.jpg         2    0.3808    0.1575  0.0483   
9  light_01_missing_hole_04_2_256.jpg         2    0.2600    0.4242  0.0600   

   height  
0  0.0550  
1  0.0617  
2  0.0600  
3  0.0667  
4  0.0567  
5  0.0433  
6  0.0367  
7

In [43]:
import os
import cv2
import numpy as np
import pandas as pd

#Set up image folder
train_images_path = "C:/Users/wenwe/Desktop/Lab_QC/Wendy/pcb-defect-dataset/train/images"
val_images_path = "C:/Users/wenwe/Desktop/Lab_QC/Wendy/pcb-defect-dataset/val/images"
test_images_path = "C:/Users/wenwe/Desktop/Lab_QC/Wendy/pcb-defect-dataset/test/images"

# Read tag CSV
train_labels_csv = "train_labels.csv"
val_labels_csv = "val_labels.csv"
test_labels_csv = "test_labels.csv"

# Define functions to read images and match tags
def load_images_and_labels(images_path, labels_csv):
    df = pd.read_csv(labels_csv)  # 讀取 CSV 標籤

    # 合併相同 image_name，取出最常出現的 class_id（單一標籤）
    df = df.groupby("image_name")["class_id"].agg(lambda x: x.value_counts().index[0]).reset_index()

    X, y = [], []

    for _, row in df.iterrows():
        img_path = os.path.join(images_path, row["image_name"])

        if os.path.exists(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)   # 灰階
            img = cv2.resize(img, (64, 64)) / 255.0             # Resize + normalize
            X.append(img)
            y.append(row["class_id"])  # 注意這裡只剩下唯一 class_id

    # 轉成 NumPy
    X = np.array(X)
    y = np.array(y)

    print(f"{labels_csv} processed, image data size: {X.shape}, label data size: {y.shape}")
    return X, y

# read `train`、`val`、`test` data
X_train, y_train = load_images_and_labels(train_images_path, train_labels_csv)
X_val, y_val = load_images_and_labels(val_images_path, val_labels_csv)
X_test, y_test = load_images_and_labels(test_images_path, test_labels_csv)

train_labels.csv processed, image data size: (6370, 64, 64), label data size: (6370,)
val_labels.csv processed, image data size: (802, 64, 64), label data size: (802,)
test_labels.csv processed, image data size: (829, 64, 64), label data size: (829,)


In [49]:
X_train = X_train[:3500]
y_train = y_train[:3500]

In [51]:
# Choose label=0&1
X_train_0 = X_train[y_train == 0]
y_train_0 = y_train[y_train == 0]

X_train_1 = X_train[y_train == 1]
y_train_1 = y_train[y_train == 1]

X_val_0 = X_val[y_val == 0]
y_val_0 = y_val[y_val == 0]

X_val_1 = X_val[y_val == 1]
y_val_1 = y_val[y_val == 1]

X_test_0 = X_test[y_test == 0]
y_test_0 = y_test[y_test == 0]

X_test_1 = X_test[y_test == 1]
y_test_1 = y_test[y_test == 1]

print(f"label=0 訓練數據數量: {X_train_0.shape}")
print(f"label=0 訓練數據數量: {X_val_0.shape}")
print(f"label=0 訓練數據數量: {X_test_0.shape}")

print(f"訓練資料 label=1 數量: {X_train_1.shape}")
print(f"驗證資料 label=1 數量: {X_val_1.shape}")
print(f"測試資料 label=1 數量: {X_test_1.shape}")


label=0 訓練數據數量: (612, 64, 64)
label=0 訓練數據數量: (140, 64, 64)
label=0 訓練數據數量: (131, 64, 64)
訓練資料 label=1 數量: (568, 64, 64)
驗證資料 label=1 數量: (130, 64, 64)
測試資料 label=1 數量: (138, 64, 64)


In [61]:
import numpy as np
import pennylane as qml

# 基本裝置設定（每個 patch 用 4 qubits）
dev = qml.device("default.qubit", wires=4, shots=None)

@qml.qnode(dev)
def quantum_kernel_with_pooling(x, theta_1=0.3, theta_2=0.6):
    for i in range(4):
        qml.RY(x[i] * np.pi, wires=i)
        qml.Hadamard(wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.RY(theta_1, wires=1)
    qml.RY(theta_2, wires=3)
    return [
        qml.expval(qml.PauliZ(0)),
        qml.expval(qml.PauliX(0)),
        qml.expval(qml.PauliZ(2)),
        qml.expval(qml.PauliX(2)),
    ]

def quanvolution_with_pooling(image, theta_1=0.3, theta_2=0.6):
    h, w = image.shape
    q_image = np.zeros((h // 2, w // 2, 4))
    for i in range(0, h, 2):
        for j in range(0, w, 2):
            patch = image[i:i+2, j:j+2].flatten()
            q_image[i//2, j//2] = quantum_kernel_with_pooling(patch, theta_1, theta_2)
    return q_image

def qcnn_with_pooling(image, theta_layers):
    for theta_1, theta_2 in theta_layers:
        image = quanvolution_with_pooling(image, theta_1, theta_2)
        h, w, c = image.shape
        image = image.reshape(h, w * c)  # 變成下一層輸入格式
    return image.flatten()

In [58]:
# 測試
img = np.random.rand(8, 8)
theta_layers = [(0.3, 0.6), (0.8, 0.4)]  # 二層
features = qcnn_with_pooling(img, theta_layers)
print("特徵維度：", features.shape)
print("特徵向量：", features)

特徵維度： (64,)
特徵向量： [ 9.87864420e-01 -6.09228990e-02  8.49576784e-02  8.89649551e-01
  1.38362398e-01 -5.14649853e-01  3.20886728e-01  6.73640810e-01
  1.04083524e-01 -9.10775496e-01  2.91744244e-01 -7.31678987e-01
  6.51040668e-03 -9.82793384e-01  9.89269849e-01 -1.37641723e-01
  9.59293091e-01 -9.21522256e-02  6.87107687e-01  7.14802083e-01
  3.82721143e-03 -9.89410820e-01  9.98662001e-01 -4.03202787e-02
  4.58622766e-02 -8.59562835e-01  6.68103233e-01  6.21106635e-02
  9.92852036e-01 -1.17444741e-01  5.96585210e-01 -6.51284933e-02
  8.76081243e-01 -4.73739406e-01  3.18207884e-01 -9.34802102e-01
  4.44196225e-01  5.85156041e-02  6.45564972e-01 -4.82839413e-01
  9.99567894e-01 -6.36225967e-03  3.25772238e-01 -6.77286654e-01
  8.97100350e-01 -4.37859186e-01  1.78980385e-01 -5.27480457e-01
  3.67114631e-01 -2.91293516e-01  1.42555593e-02 -9.66272512e-01
  7.68626862e-01  3.17057986e-01  2.33618322e-01 -8.90863614e-01
  7.77303330e-01  3.58183593e-01  1.28367003e-01 -8.64446266e-01
  8.603

# ｔｅｓｔ　

In [12]:
import pennylane as qml
import numpy as np

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits, shots=1000)  # 多次測量以模擬現實

@qml.qnode(dev)
def create_quantum_kernel_with_param_pooling():
    @qml.qnode(dev)
    def circuit(x, theta_1, theta_2):
        for i in range(n_qubits):
            qml.RY(x[i] * np.pi, wires=i)
            qml.Hadamard(wires=i)

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])
        qml.RY(theta_1, wires=1)
        qml.RY(theta_2, wires=3)

        return [
            qml.expval(qml.PauliZ(0)),
            qml.expval(qml.PauliX(0)),
            qml.expval(qml.PauliZ(2)),
            qml.expval(qml.PauliX(2)),
        ]
    return circuit

# x_test = np.random.rand(4)  # 產生 4 個隨機數
# qml.draw_mpl(quantum_kernel)(x_test)

In [13]:
def quanvolution_with_param_pooling(image, theta_values):
    output_shape = (image.shape[0] // 2, image.shape[1] // 2, 4)
    q_image = np.zeros(output_shape)

    kernel_circuit = create_quantum_kernel_with_param_pooling()

    for i in range(0, image.shape[0], 2):
        for j in range(0, image.shape[1], 2):
            patch = image[i:i+2, j:j+2].flatten()
            q_features = kernel_circuit(patch, theta_values[0], theta_values[1])
            q_image[i // 2, j // 2] = q_features

    return q_image.flatten()

In [14]:
def qcnn_param(image, theta_layers):
    for theta in theta_layers:
        image = image.reshape(int(np.sqrt(len(image))), -1)
        image = quanvolution_with_param_pooling(image, theta)
    return image

In [15]:
import numpy as np

sample_image = np.random.rand(8, 8)
theta_layers = [[0.3, 0.6], [0.8, 0.4]]  # 每層的池化參數
features = qcnn_param(sample_image, theta_layers)

print("最終特徵維度：", features.shape)
print("量子特徵向量：", features)

QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.

# Data qcnn

In [63]:
theta_layers = [(0.3, 0.6), (0.8, 0.4)]

In [72]:
X_train_0 = np.array([qcnn_with_pooling(img, theta_layers) for img in X_train_0])
X_val_0 = np.array([qcnn_with_pooling(img, theta_layers) for img in X_val_0])
X_test_0 = np.array([qcnn_with_pooling(img, theta_layers) for img in X_test_0])

print(" X_train_0 shape:", X_train_0.shape)  # (84, 特徵數)
print(" X_val_0 shape:", X_val_0.shape)      # (88, 特徵數)
print("X_test_0 shape:", X_test_0.shape)    # (79, 特徵數)

 X_train_0 shape: (612, 4096)
 X_val_0 shape: (140, 4096)
X_test_0 shape: (131, 4096)


In [73]:
X_train_1 = np.array([qcnn_with_pooling(img, theta_layers) for img in X_train_1])
X_val_1 = np.array([qcnn_with_pooling(img, theta_layers) for img in X_val_1])
X_test_1 = np.array([qcnn_with_pooling(img, theta_layers) for img in X_test_1])

print(" X_train_1 shape:", X_train_1.shape)  # (84, 特徵數)
print(" X_val_1 shape:", X_val_1.shape)      # (82, 特徵數)
print("X_test_1 shape:", X_test_1.shape)    # (84, 特徵數)

 X_train_1 shape: (568, 4096)
 X_val_1 shape: (130, 4096)
X_test_1 shape: (138, 4096)


In [76]:
# 合併 label=0 和 label=1 的量子特徵數據
X_train_01 = np.vstack([X_train_0, X_train_1])
y_train_01 = np.hstack([y_train_0, y_train_1])

print(f"合併後的訓練數據: {X_train_01.shape}")
print(f"合併後的標籤數據: {y_train_01.shape}")

X_val_01 = np.vstack([X_val_0, X_val_1])
y_val_01 = np.hstack([y_val_0, y_val_1])

print(f"合併後的訓練數據: {X_val_01.shape}")
print(f"合併後的標籤數據: {y_val_01.shape}")

X_test_01 = np.vstack([X_test_0, X_test_1])
y_test_01 = np.hstack([y_test_0, y_test_1])

print(f"合併後的訓練數據: {X_test_01.shape}")
print(f"合併後的標籤數據: {y_test_01.shape}")

合併後的訓練數據: (1180, 4096)
合併後的標籤數據: (1180,)
合併後的訓練數據: (270, 4096)
合併後的標籤數據: (270,)
合併後的訓練數據: (269, 4096)
合併後的標籤數據: (269,)


In [78]:
np.save("qcnn_features_X_train_01p.npy", X_train_01)
np.save("qcnn_labels_y_train_01p.npy", y_train_01)

np.save("qcnn_features_X_val_01p.npy", X_val_01)
np.save("qcnn_labels_y_val_01p.npy", y_val_01)

np.save("qcnn_features_X_test_01p.npy", X_test_01)
np.save("qcnn_labels_y_test_01p.npy", y_test_01)